In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
from os import listdir
from os.path import splitext
import ipywidgets as widgets
from IPython.display import display
from dependencies import auto_correlation, cross_correlation
%matplotlib widget

In [2]:
def on_change_select_audio(change):
    with out:
        print(change)
        file_extension = splitext(change["options"][change["new"]["index"]])[1]
        if (file_extension == ".txt") or (file_extension == ".wav"):
            print(change["new"], "ends in", file_extension)

In [3]:
print("Please select only audio wav and txt extension from menu")
files = listdir(Path().absolute())

select_audio = widgets.Select(
    options=files,
    description='Select Audio:',
    disabled=False
)
out = widgets.Output()

display(select_audio, out)
select_audio.observe(on_change_select_audio)

Please select only audio wav and txt extension from menu


Select(description='Select Audio:', options=('dependencies.py', '.ipynb_checkpoints', 'Bahan Study Kasus - Lin…

Output()

In [4]:
print(widgets.Select.observe.__doc__)

Setup a handler to be called when a trait changes.

        This is used to setup dynamic notifications of trait changes.

        Parameters
        ----------
        handler : callable
            A callable that is called when a trait changes. Its
            signature should be ``handler(change)``, where ``change`` is a
            dictionary. The change dictionary at least holds a 'type' key.
            * ``type``: the type of notification.
            Other keys may be passed depending on the value of 'type'. In the
            case where type is 'change', we also have the following keys:
            * ``owner`` : the HasTraits instance
            * ``old`` : the old value of the modified trait attribute
            * ``new`` : the new value of the modified trait attribute
            * ``name`` : the name of the modified trait attribute.
        names : list, str, All
            If names is All, the handler will apply to all traits.  If a list
            of str, handler wil